# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

## <b>Исследование объявлений о продаже квартир:</b> 
Задача исследования представляет собой установление параметров определения рыночной стоимости объектов недвижимости. 
Нам необходимо превести обработку данных, посчитать дополнительные критерии оценки и провести исследовательский анализ данных по группе параметров.
    
Данное исследование разделим на несколько частей:
    
### Часть 1. Изучение общей информации
    
### Часть 2. Предобработка данных
    
### Часть 3. Подсчёт новых значений
    
###  Часть 4. Исследовательский анализ данных

### Часть 5. Общий вывод

### Шаг 1. Откройте файл с данными и изучите общую информацию.
<a id='section1'></a>

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
df.info()
display(df.head(10))

for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100))) #посчитаем процент пропусков для наглядности
    

### Вывод

Всего в таблице 22 столбца, 14 пренадлежат типу float, 3 - int, 3 - object, 2 - bool. Не все названия столбцов корректны. 
Пропуски данных обнаружены в стобцах ceiling_height, living_area, is_apartment, kitchen_area, balcony, airports_nearest, cityCenters_nearest, parls_around3000, parks_nearest, ponds_Around3000, ponds_nearest, days_exposition. 
Для последующего анализа нам понадобятся значения площади, цены, числа комнат, высоты потолков, удаленности от центра, количества этажей, даты размещения и времени продажи.


### Шаг 2. Предобработка данных
<a id='section2'></a>

### Переименование столбцов


In [ ]:
df = df.rename(columns={"cityCenters_nearest": "city_center_distance", "studio": "is_studio", "parks_around3000": "parks_around_3000", "ponds_around3000": "ponds_around_3000","open_plan": "is_open_plan", "ponds_nearest": "ponds_distance", "rooms":"rooms_number", "studio":"is_studio"})
df.columns #проверка

#### Вывод

Замена была произведена в 8 стобцах: столбцам были подобраны более говорящие названия и снижен регистр, что позволило упростить последующую работу с данными.


### Обработка пропусков


In [ ]:
df['balcony'] = df['balcony'].fillna(0) #заполнение пропусков нулевым значением
display(df['balcony'].sample(10)) #проверка по случайным значениям

#### Вывод

Пропуски в данных о наличии балкона могут свидетельствовать об их отсутствии, поэтому заполняем их нулевым значением.

In [ ]:
df.loc[df['ceiling_height'].isna(), 'ceiling_height'] = df['ceiling_height'].median() #заполнение пропусков медианой



#### Вывод

В данных о высоте потолков очень много пропусков - 39%. О таком высоком значении стоило бы сообщить команде, занимающейся сбором данных. Как вариант, эту графу в заполнении данных о квартире можно было сделать обязательной, чтобы её не пропускали пользователи.
Так как у нас нет данных, из которых можно было бы высчитать высоту потолков, заполним эти пропуски медианой - т.к. она более устойчива к выбросам по отношению к среднему значению.

In [ ]:
df.loc[df['kitchen_area'].isna(), 'kitchen_area'] = df['kitchen_area'].median() #заполнение пропусков медианой
display(df['kitchen_area'].isna().sum(), df['kitchen_area'].sample(10)) #проверка 
n = df['total_area'] - df['kitchen_area'] #посчитаем разницу общей площади и кухонной площади
df.loc[df['living_area'].isna(), 'living_area'] = df['living_area'].fillna(n) #заполним отсутствующие значения разницей
display(df['living_area'].isna().sum(), df['living_area'].sample(10)) #проверка



#### Вывод

Значения пропусков площади кухни заполним медианой.
Пропуски в значения жилой площади заполним разницей между общей площадью и площадью кухни.

In [ ]:
df.dropna(subset = ['locality_name'], inplace = True) #удалим пропущенные значения города, сохранив индексы
df['locality_name'] = df['locality_name'].str.lower() #снизим регистр

df.loc[(df['city_center_distance'].isna())&(df['locality_name'] == 'санкт-петербург'), 'city_center_distance'] = \
df.loc[df['locality_name'] == 'санкт-петербург', 'city_center_distance'].median() #заполним пропуски в значениях отдалённости от центра в Питере медианой
df.loc[(df['city_center_distance'].isna())&(df['locality_name'] != 'санкт-петербург'), 'city_center_distance'] = \
df.loc[df['locality_name'] != 'санкт-петербург', 'city_center_distance'].median() #заполним пропуски в значениях отдалённости от центрав других городах медианой
display(df['city_center_distance'].isna().sum(), df['city_center_distance'].sample(10)) #проверка

#### Вывод

Пропусков в значения локации меньше 1% от данных, поэтому их удаление не должно повлиять на последующий анализ, удаляем, сохранив индексы. Также приведём значения к нижнему регистру, для удобства последующей обработки.
Пропуски в данных об удалённости от центра заполним медианой, до этого разделив значения на Питер и другие города, т.к. значения в большом городе могут отличаться.

In [ ]:
df.dropna(subset = ['floors_total'], inplace = True) #удаляем значения, сохранив индекс

#### Вывод

Пропусков в значения количества этажей меньше 1% от данных, поэтому их удаление не должно повлиять на последующий анализ, удаяем, сохранив индексы.

### Замена типа данных


In [ ]:
df['floors_total'] = df['floors_total'].astype('int') #заменяем значения с плавающей точкой на целочисленной
df['is_apartment'] = df['is_apartment'].astype('bool') #приводим к булевому значению
#df['days_exposition'] = df['days_exposition'].astype('int')
df['balcony'] = df['balcony'].astype('int')
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S') #приводим к формату даты и времени
df.info() #проверка


#### Вывод

Значения столбцов с количеством этажей, балконов и дней размещения приводим к целочисленному типу, т.к. здесь значения после запятой не нужны.
Значения в колонке с типом апартаментов булевые, к этому типу и приводим.
Также приводим дату публикации к формату даты и времени.

In [ ]:
df.drop(['airports_nearest', 'parks_around_3000', 'parks_nearest', 'ponds_around_3000', 'ponds_distance'], axis='columns', inplace=True)
df.info()

#### Вывод

Значения столбцов с аэропортами, парками и прудами в последующем анализе нам не понадобятся, убираю их из дата фрейма, чтобы не мешались:)

### Шаг 3. Посчитайте и добавьте в таблицу
<a id='section3'></a>

In [ ]:
#цену квадратного метра;
#день недели, месяц и год публикации объявления;
#этаж квартиры; варианты — первый, последний, другой;
#соотношение жилой и общей площади, а также отношение площади кухни к общей.

df['price_square'] = df['last_price']/df['total_area']
df['weekday'] = df['first_day_exposition'].dt.weekday
df['mounth'] = pd.DatetimeIndex(df['first_day_exposition']).month 
df['year'] = pd.DatetimeIndex(df['first_day_exposition']).year 
df['floor_category'] = pd.Series(index=df.index)
df.loc[df['floor'] == 1, 'floor_category'] = 'Первый'
df.loc[df['floor'] == df['floors_total'], 'floor_category'] = 'Последний'
df.loc[(df['floor'] != df['floors_total'])&(df['floor'] != 1), 'floor_category'] ='Другой'
df['living_ratio'] = df['living_area']/df['total_area']
df['kitchen_ratio'] = df['kitchen_area']/df['total_area']
df.info()

#### Вывод

Подсчитаны новые данные и добавлены новые колонки в дата фрейм.

### Шаг 4. Проведите исследовательский анализ данных и выполните инструкции:
<a id='section4'></a>

In [ ]:
import plotly.express as px

def hist_for_column(data, column, name): #функция построения гистограммы
    fig = px.histogram(
    data,
    x = column,
    title = name)
    print(data[column].describe())
    fig.show()

In [ ]:
hist_for_column(df, 'total_area', 'Total area')

fig = px.histogram(df, x="total_area", color="rooms_number")
fig.show()

In [ ]:
hist_for_column(df, 'last_price', 'Price')



In [ ]:
xlim = [0, 100000000]
fig = px.histogram(
df,
histnorm='probability density',
x = 'last_price',
title = 'Price',
range_x = [0, 100000000])



fig.show()

In [ ]:
hist_for_column(df, 'ceiling_height', 'Ceiling height')

In [ ]:
hist_for_column(df, 'rooms_number', 'Rooms number')

#### Вывод

Построено четыре графика с использованием библиотеки plotly, т.к. она позволяет строить красивые качественные графики с возможностью последующего их изучения при наведении курсора и зума. 
Все графики имеют распределение близкое к Пуассону.
1 график total area отражает колличество квартир и их общую площадь. На графике ярко заметны несколько пиков. Предполагаем, что их наличие может быть связано с числом комнат в квартире. Строим ещё один график, где цветом отражено распределение по количеству комнат. Действительно, первый пик относится к однокомнатным квартирам,второй и третий - к двухкомнатным (можно сделать вывод о влиянии и других факторов на данные о площади). Также можно сразу выделить большое количество трёхкомнатных квартир. Средняя площадь квартир составляет 60 метров квадратных и большая часть таких значений принадлежит к двухкомнатным, в меньшей степени - трёхкомнатным и в минимальной - однокомнатным. В целом значения распределени от 12 до 200 метров квадратных, квартиры с большей площадью встречаются редко.
2 график price отражает количество квартир и их стоимость. Здесь мы можем наблюдать унимодальный график с ярковыраженным пиком. Можно сделать вывод о выбросе в данных, т.к. большое количество квартир находятся в ценовом диапазоне от 3,5 до 4 млн, что отражает медиана (=4,6 млн), а среднее значение из-за выбросов искажено и составляет 6 млн. Значения лежат в диапазоне от 1,3 до 40 млн, более высокие значения встречаются редко.
3 график ceiling height отражает количество квартир и высоту потолков в них. Среднее значение и медиана близки, в большей части квартир высота потолка составляет 2,6 м, это также отражено ярким пиком. Значений меньшей величины очень мало, что в целом очевидно, так как в квартире с потолком ниже 2,5 метра жить сложно. Потолки выше 4 метров встречаются редко.
4 график отражает количество комнат. В среднем в данных представлены двухкомнатные квартиры. Данные в среднемраспределены в значениях от нуля (это могут быть студии, апартаменты или квартиры со свободной планировкой) до 8-комнатных квартир. Более высокие значения встречаются редко.

In [ ]:
hist_for_column(df, 'days_exposition', 'Days exposition')

#### Вывод

Значения медианы и среднего значения разительно отличаются. Можно сделать вывод о наличии выбросов в данных. На графике заметные характерные пики, большая часть квартир была продана за 90-99 дней, 40-49 дней и 60-69 дней. Больше 100 дней квартиры продаются реже - значения на графике снижаются, квартира продаётся медленно. Больше 800 - большая редкость. Продажу за срок до 40 дней можно считать быстрой. 

In [ ]:
def box_plot(data, column, name): 
    print(data[column].describe())
    fig = px.box(data, y = column,  title = name)
    fig.show()
# функция вывода ящика с усами

In [ ]:
box_plot(df, 'total_area', 'Total area')

q1 = df.total_area.quantile(0.25) #считаем 1 квантиль
q3 = df.total_area.quantile(0.75) #считаем 3 квантиль
iqr =q3-q1 #межквартильный размах
df_new = df.query('total_area <= @q3+1.5*@iqr')

box_plot(df_new, 'total_area', 'Total area new')


#### Вывод

По ящику с усами обнаруживаем большое количество выбросов. Пробуем их убрать, построив лимиты по межквартильному размаху. Теперь выглядит красиво и без ярких выбросов (те, что имеются, связаны с тем, что сместилась медиана).

In [ ]:
box_plot(df, 'ceiling_height', 'Ceiling height')

df_new = df_new.query('2.4 <= ceiling_height <= 4')

box_plot(df_new, 'ceiling_height', 'Ceiling height new')


#### Вывод

Минимальной нормой в России считается высота потолка 2,4 метра, в наших данных встречаются более низкие значения, их стоит убрать. Также убираем значения с высотой потолка выше 4 метров, т.к. такие значения встречаются редко. 

In [ ]:
box_plot(df, 'last_price', 'Price')


df_new = df_new.query('last_price <= 40000000')

box_plot(df_new, 'last_price', 'Price new')


#### Вывод

Убираем значения стоимости выше 40 млн. - т.к. они встречаются редко.


In [ ]:
box_plot(df, 'rooms_number', 'Rooms number')

q1 = df.rooms_number.quantile(0.25)
q3 = df.rooms_number.quantile(0.75)
iqr =q3-q1
df_new = df_new.query('rooms_number <= @q3+1.5*@iqr')

box_plot(df_new, 'rooms_number', 'Rooms number new')


#### Вывод

На графике обнаруживаем небольшое кол-во выбросов - убираем их с помощью подчёта межквартильного размаха.


In [ ]:
box_plot(df, 'days_exposition', 'Days exposition')

df_new = df_new.query('days_exposition <= 800')

box_plot(df_new, 'days_exposition', 'Days exposition new')

#### Вывод

Значения со сроком размещения публикации, превышающие 800 дней, всречаются очень редко, поэтому убираем их.


In [ ]:
print("В итоге отрезали {:.1%} данных".format((len(df)-len(df_new))/len(df)))

#### Вывод

По итогу проведённой работы по обработке редких и выбивающихся значений мы отрезали 7.7% данных. Думаю, что это достаточно большой процент и всё же приемлемый, дальнейшую работу проводим с новым df - df_new.

In [ ]:
print("Корреляция цены и столбцов:")
print(df_new[['total_area', 'rooms_number', 'city_center_distance']].corrwith(df_new['last_price']))
columns = ['total_area', 'rooms_number', 'city_center_distance','last_price']
fig = px.scatter_matrix(df_new[columns], title = 'Scatter Matrix')
fig.show()

#### Вывод

По результатам корреляции Пирсона мы наблюдаем прямую взаимосвязь площадь квартиры и её стоимости, а также обратную корреляцию удалённости от центра и стоимости (чем дальше от центра находится квартира - тем ниже её стоимость, что в целом логично). Цена также прямо коррелирует с количеством комнат, но в меньшей степени.
Данные выводы иллюстрируются диаграммами на матрице рассеяния.

In [ ]:
fig = px.histogram(df_new, x="floor_category", y = 'last_price', color = 'floor_category', histfunc = 'avg', title = 'Correlation between price and floor category')
fig.show()

In [ ]:
df_new.columns 
fig = px.histogram(df_new, nbins = 10, x="weekday", y = 'last_price', color = 'weekday', histfunc = 'avg', title = 'Correlation between price and weekday')
fig.show()

fig = px.histogram(df_new, x="mounth", y = 'last_price', color = 'mounth', histfunc = 'avg', title = 'Correlation between price and weekday')
fig.show()

fig = px.histogram(df_new, x="year", y = 'last_price', color = 'year', histfunc = 'avg', title = 'Correlation between price and weekday')
fig.show()

#### Вывод

Квартиры на последем этаже в среднем дороже, чем на первом.

Взаимосвязи стоимости квартиры и дня недели/месяца публикации не наблюдается.
Видим определённую зависимость стомости от года. С 2014 по 2015 год происходит резкое падение стоимости, практически на 1.5 млн. Возможно, это связано с общим кризисом и ослаблением рубля. в 2016-2018 году стоимость держится примерно на одном уровне, а в 2019 начинает расти. 

In [ ]:
df_new.columns
data = df_new['locality_name'].value_counts().head(10) #выбираем 10 локаций с самым высоким числом объявлений
sorted_city_list = data.index
df_new.query('locality_name in @sorted_city_list').pivot_table(index='locality_name', values='price_square').sort_values(by='price_square', ascending=False)
#строим сводную таблицу по стоимости

#### Вывод

Населённый пункт с самой высокой стоимостью жилья - Санкт-Петербург, самой низкой - Выборг.

In [ ]:
import warnings

warnings.filterwarnings('ignore')

df_sp = df_new.query('locality_name == "санкт-петербург"') #выбираем квартиры в Питере
df_sp['city_center_distance_km'] = (df_sp['city_center_distance'] / 1000).round()

fig = px.histogram(df_sp, x='city_center_distance_km', y = 'last_price',  histfunc = 'avg', title = 'Price and city center distance') #строим гистограмму удалённости от центра и среднего значения стоимости
fig.show()


#### Вывод

На графике можно выделить резкое падение средней стоимости после 7  км удалённости от центра, по этому значению и определим центр.
Также наблюдаем странный пик с высокой стоимостью в 27 км от центра, как вариант, это может быть район с элитной загородной недвижимостью.

In [ ]:
df_sp_not_center = df_sp.query('city_center_distance_km > 7')
df_sp_center = df_sp.query('city_center_distance_km <= 7') 

hist_for_column(df_sp_center, 'total_area', 'Total area center')
hist_for_column(df_sp_not_center, 'total_area', 'Total area not center')


#### Вывод

Сравнивая площадь квартир в центре Петербурга и за центром можно сделать вывод, что в среднем площадь квартир в центре выше.В центре средние значения лежат в диапазоне от 50 до 86 квадратных метров, а за центром - от 39 до 64.

In [ ]:
hist_for_column(df_sp_center, 'last_price', 'Total area center')
hist_for_column(df_sp_not_center, 'last_price', 'Total area not center')

#### Вывод

Сравнивая стоимость квартир в центре Петербурга и за центром можно сделать вывод, что в среднем площадь квартир в центре значительно выше. В центре средние значения лежат в диапазоне от 6 до 10 млн, а за центром - от 4 до 6,5 млн.

In [ ]:
hist_for_column(df_sp_center, 'rooms_number', 'Rooms number')
hist_for_column(df_sp_not_center, 'rooms_number', 'Rooms number not center')

#### Вывод

Сравнивая кол-во комнат в квартирах в центре Петербурга и за центром можно сделать вывод, что в среднем среди квартир в обоих случаях преобладают двухкомнатные. Но в центре больше 4 и 5 комнатных квартир, чем за центром, а за центром значительно больше 1-комнатных.

In [ ]:
hist_for_column(df_sp_center, 'ceiling_height', 'Ceiling height')
hist_for_column(df_sp_not_center, 'ceiling_height', 'Ceiling height not center')

#### Вывод

Сравнивая высоту потолков в квартирах в центре Петербурга и за центром можно сделать вывод, что в среднем потолки в квартирах в центре выше, средний диапазон значений - от 2,6 до 3, в отличие от квартир за центром - от 2,6 до 2,7, также в центре чаще встречаются значения выше 3 метров.

In [ ]:
print("Корреляция цены и столбцов:")
print(df_sp_center[['total_area', 'rooms_number', 'city_center_distance']].corrwith(df_new['last_price']))
print(df_sp_not_center[['total_area', 'rooms_number', 'city_center_distance']].corrwith(df_new['last_price'])) 

#### Вывод

По значения корреляции Пирсона можно сделать вывод о том, что зависимость цены от площади квартир вне центра выше, нежели квартир в центре, но в обоих случаях корреляция есть и она положительная.
То же происходит и с зависимостью цены от количества комнат - корелляция вне центра выше.
Корреляции цены и удалённости от центра не наблюдается, т.к. наши данные уже распределены по группам относительно этого параметра.

In [ ]:
def hist_sp(data):
    fig = px.histogram(
    data,
    x = 'weekday',
    y = 'last_price',
    color = 'weekday',
    histfunc = 'avg',
    title = 'Correlation between price and weekday')
    
    fig.show()
    
    fig = px.histogram(
    data,
    x = 'mounth',
    y = 'last_price',
    color = 'mounth',
    histfunc = 'avg',
    title = 'Correlation between price and mounth')
    
    fig.show()
    
    fig = px.histogram(
    data,
    x = 'year',
    y = 'last_price',
    color = 'year',
    histfunc = 'avg',
    title = 'Correlation between price and year')
    
    fig.show()
    


In [ ]:
print('Correlation between date and price in center of SPb:')
hist_sp(df_sp_center)
print('Correlation between date and price out of center of SPb:')
hist_sp(df_sp_not_center)

#### Вывод

В обоих случаях особой взаимосвязи между днём и месяцом публикации и средней стоимости нет.
Интересная взаимосвязь наблюдается с годом публикации - в среднем  стоимость на квартиры в центре с 14 по 16 год растёт, в 2017 падает до значения 2015 года и снова начинает возрастать. 
А самое высокое значение средней стоимости квартир за центром наблюдалась в 2014 году, после чего резко снизилась, и с 2017 года понемногу растёт.


### Шаг 5. Общий вывод
<a id='section5'></a>

Типичная квартира по данным всего датасета является двухкомнатной, площадью 60 кв.м., с высотой потолка 2,6 метра. Стоит такая кватира 5 млн, а продаётся за 144 дня.
На стомость ей влияет площадь (прямая корреляция), удалённость от центра (дальше - дешевле), и количество комнат (прямая). Квартира на первом этаже в среднем будет стоить дешевле, нежели на последнем.
Да стоимость не влияет день недели и месяц. Самая высокая средняя стоимость наблюдалась в 2014 году, после чего снизилась и возросла в 2019 году и составила 5.6 млн.

Типичная квартира в центре Петербурга является двухкомнатной, площадью 68 кв.м., с высокими потолками - 2,9 м. и стоимостью 9 млн. Начиная с 2017 года стомость квартиры в центре увеличивается.

Типичная квартира вне центра Петербурга является двухкомнатной, площадью 53 кв.м., с высотой потолка 2.6 м. и стоиостью 5,7 млн.

В целом квартиры в центре Петербурга отличаются более высокой стомостью, большей площадью и большей высотой потолков. Это может быть связано с большим количеством в центре исторических квартир старого образца, тем временем как вне центра появляются более типичные современности новостройки.